In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.metrics.pairwise import cosine_similarity

from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [10]:
# load model
model = tf.keras.models.load_model('/content/gdrive/My Drive/data/stereo_matching/model_shared_weight.h5')
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, None, None, 3 56704       input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, None, None, 6 0           sequential[0][0]             

In [12]:
left  = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_left_1.png", 0)
right = cv2.imread("/content/gdrive/My Drive/data/stereo_matching/Aloe_right_2.png", 0)

#z-score
mean = np.mean(left)
std  = np.std(left)
left = (left - mean)/(std+1e-7)

mean = np.mean(right)
std  = np.std(right)
right = (right - mean)/(std+1e-7)

disparity_range = 64  # D
H, W = right.shape

print(left.shape)
print(right.shape)
print(H, W)

(370, 427)
(370, 427)
370 427


In [14]:
cost_volume = np.zeros((356, 413, disparity_range))

for loc in range(disparity_range):
    x_off = -loc
    l = left[:, max(0, -x_off):W]
    r = right[:, 0:min(W, W + x_off)]
    
    l = tf.expand_dims(l, axis=0)
    l = tf.expand_dims(l, axis=3)

    r = tf.expand_dims(r, axis=0)
    r = tf.expand_dims(r, axis=3)
    
    similarity_map = model([l, r])
    cost_volume[:, loc:413, loc] = similarity_map[:, :]

In [16]:
from PIL import Image

pred = tf.argmax(cost_volume, axis=2)
pred = pred.numpy() * 3.0

# save
pred = pred.astype(np.uint8)
im = Image.fromarray(pred)
im.save('/content/gdrive/My Drive/data/stereo_matching/disparity_shared_weight.png')